In [7]:
# Parámetros dados
p_hat = 0.87
alpha = 0.0125
n = 1000

# Calcular el valor crítico Z
z_critical = norm.ppf(1 - alpha/2)

# Calcular el error estándar
std_error = math.sqrt((p_hat * (1 - p_hat)) / n)

# Calcular el intervalo de confianza
lower_limit = p_hat - z_critical * std_error

# Imprimir el resultado redondeado a cuatro decimales
print(f"El límite inferior del intervalo de confianza es: {lower_limit:.4f}")


El límite inferior del intervalo de confianza es: 0.8434


In [4]:
from scipy.stats import norm
import math

alpha = 0.0125
z = norm().ppf(1 - alpha / 2)
p = 0.87
N = 1000
Cn = (p - z * np.sqrt(p * (1 - p) / N),
      p + z * np.sqrt(p * (1 - p) / N))
Cn

(0.8434372809894305, 0.8965627190105695)

# 2

In [2]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score
from scipy.stats import wilcoxon
import numpy as np


In [27]:
K = 30
kf = StratifiedKFold(n_splits=K, random_state=0, shuffle=True)
X, y = load_breast_cancer(return_X_y=True)

P = []
for tr, ts in kf.split(X, y):
    forest = RandomForestClassifier().fit(X[tr], y[tr]).predict(X[ts])
    extra_trees = ExtraTreesClassifier().fit(X[tr], y[tr]).predict(X[ts])
    P.append([recall_score(y[ts], hy, average="macro") for hy in [forest, extra_trees]])
P = np.array(P)


In [28]:
p = P[:, 0] - P[:, 1]
t = np.sqrt(K) * np.mean(p) / np.std(p)

In [29]:
p_values = wilcoxon(P[:, 0], P[:, 1]).pvalue

# Define el nivel de significancia
alpha = 0.05

# Compara el p-value con el nivel de significancia
result = p_values > alpha

# Imprime el resultado
print("Los algoritmos son estadísticamente similares:", result)


Los algoritmos son estadísticamente similares: True


/Users/aaron/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [30]:
wilcoxon(P[:, 0], P[:, 1])

WilcoxonResult(statistic=30.0, pvalue=0.7881362654668115)

In [11]:


# Parámetros
K = 30
kf = StratifiedKFold(n_splits=K, random_state=0, shuffle=True)
X, y = load_breast_cancer(return_X_y=True)

# Inicializar matrices
forest = np.empty_like(y)
extra_trees = np.empty_like(y)

# Entrenar y predecir con RandomForestClassifier y ExtraTreesClassifier
for tr, ts in kf.split(X, y):
    forest[ts] = RandomForestClassifier().fit(X[tr], y[tr]).predict(X[ts])
    extra_trees[ts] = ExtraTreesClassifier().fit(X[tr], y[tr]).predict(X[ts])

# Calcular la diferencia de rendimiento macro-F1
diff = lambda y, hy1, hy2: recall_score(y, hy1, average="macro") - recall_score(y, hy2, average="macro")

# Generar muestras aleatorias
S = np.random.randint(y.shape[0], size=(500, y.shape[0]))

# Calcular la diferencia de rendimiento para las muestras aleatorias
B = [diff(y[s], forest[s], extra_trees[s]) for s in S]

# Realizar la prueba de Wilcoxon
statistic, p_value = wilcoxon(B)

# Definir el nivel de significancia
alpha = 0.05

# Comprobar si los algoritmos son estadísticamente similares
result = p_value > alpha

# Imprimir el resultado
print("Los algoritmos son estadísticamente similares:", result)
print("Estadístico de prueba de Wilcoxon:", statistic)
print("P-valor:", p_value)


Los algoritmos son estadísticamente similares: False
Estadístico de prueba de Wilcoxon: 42199.0
P-valor: 3.208232101553484e-06


In [13]:
(np.array(B) < 0).mean()

0.398

# 3

In [34]:

from scipy.stats import norm, wilcoxon
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score
import numpy as np
import seaborn as sns

In [52]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np

# Cargar el conjunto de datos Diabetes
X, y = load_diabetes(return_X_y=True)

# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Calcular los scores \( R^2 \) usando validación cruzada
scores = cross_val_score(model, X, y, scoring='r2', cv=10)

# Calcular el intervalo de confianza con percentiles
lower_bound = np.percentile(scores, 2.5)
upper_bound = np.percentile(scores, 97.5)

# Imprimir el intervalo de confianza (redondeado a dos decimales)
print(f"Intervalo de confianza para R^2: ({lower_bound:.2f}, {upper_bound:.2f})")


Intervalo de confianza para R^2: (0.24, 0.67)


In [5]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Cargar el conjunto de datos Diabetes
X, y = load_diabetes(return_X_y=True)

# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Configurar la validación cruzada K-fold
kf = KFold(n_splits=10, random_state=0, shuffle=True)

# Lista para almacenar los scores R^2
r2_scores = []

# Calcular los scores R^2 usando validación cruzada
for tr, ts in kf.split(X, y):
    model.fit(X[tr], y[tr])
    y_pred = model.predict(X[ts])
    r2 = model.score(X[ts], y[ts])
    r2_scores.append(r2)

# Calcular el intervalo de confianza con percentiles
lower_bound = np.percentile(r2_scores, 2.5)
upper_bound = np.percentile(r2_scores, 97.5)

# Imprimir el intervalo de confianza (redondeado a dos decimales)
print(f"Intervalo de confianza para R^2: ({lower_bound:.4f}, {upper_bound:4f})")


Intervalo de confianza para R^2: (0.2909, 0.644955)


In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
import numpy as np

# Cargar datos de Diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Crear un modelo de regresión lineal
model = LinearRegression()

# Calcular R^2 utilizando validación cruzada
r2_scores = cross_val_score(model, X, y, scoring='r2', cv=10)

# Calcular el intervalo de confianza
alpha = 0.05
lower_bound = np.percentile(r2_scores, 100 * (alpha / 2))
upper_bound = np.percentile(r2_scores, 100 * (1 - alpha / 2))

# Imprimir el intervalo de confianza superior con dos decimales
print(f"El valor del intervalo superior del R^2 es: {upper_bound:.2f}")
upper_bound


El valor del intervalo superior del R^2 es: 0.67


0.6713346048507146

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_diabetes
import numpy as np

# Configurar el número de splits (K) y alpha
K = 10
alpha = 0.05

# Cargar datos de Diabetes
X, y = load_diabetes(return_X_y=True)

# Crear un modelo de regresión lineal
model = LinearRegression()

# Calcular R^2 utilizando validación cruzada
r2_scores = cross_val_score(model, X, y, scoring='r2', cv=K)

# Calcular el intervalo de confianza
lower_bound = np.percentile(r2_scores, 100 * (alpha / 2))
upper_bound = np.percentile(r2_scores, 100 * (1 - alpha / 2))

# Imprimir el intervalo de confianza superior con dos decimales
print(f"El valor del intervalo superior del R^2 es: {upper_bound:.2f}")


El valor del intervalo superior del R^2 es: 0.67


In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
import numpy as np

# Configurar el número de splits (K) y alpha
K = 10
alpha = 0.05

# Cargar datos de Diabetes
X, y = load_diabetes(return_X_y=True)

# Crear un modelo de regresión lineal
model = LinearRegression()

# Calcular R^2 utilizando validación cruzada
r2_scores = cross_val_score(model, X, y, scoring='r2', cv=K)

# Calcular el intervalo de confianza
lower_bound = np.percentile(r2_scores, 100 * (alpha / 2))
upper_bound = np.percentile(r2_scores, 100 * (1 - alpha / 2))

# Imprimir el intervalo de confianza superior con dos decimales
print(f"El valor del intervalo superior del R^2 es: {upper_bound:.2f}")


El valor del intervalo superior del R^2 es: 0.67


In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.datasets import load_diabetes
import numpy as np

# Configurar el número de splits (K) y alpha
K = 10
alpha = 0.05

# Cargar datos de Diabetes
X, y = load_diabetes(return_X_y=True)

# Crear un modelo de regresión lineal
model = LinearRegression()

# Calcular R^2 utilizando validación cruzada
y_true, y_pred = [], []

for tr, ts in KFold(n_splits=K, shuffle=True, random_state=0).split(X, y):
    model.fit(X[tr], y[tr])
    y_true.extend(y[ts])
    y_pred.extend(model.predict(X[ts]))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calcular el intervalo de confianza
n = len(y_pred)
mse = np.sum((y_true - y_pred) ** 2) / (n - 2)  # Mean Squared Error
std_error = np.sqrt(mse / np.sum((X - np.mean(X, axis=0)) ** 2))
margin_error = 1.96 * std_error / np.sqrt(n)

r2_value = r2_score(y_true, y_pred)

# Calcular el intervalo de confianza
lower_bound = r2_value - margin_error
upper_bound = r2_value + margin_error

# Asegurarse de que el intervalo esté en el rango [0, 1]
lower_bound = max(lower_bound, 0)
upper_bound = min(upper_bound, 1)

# Imprimir el intervalo de confianza superior con dos decimales
print(f"El valor del intervalo superior del R^2 es: {upper_bound:.2f}")



El valor del intervalo superior del R^2 es: 1.00


In [60]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

X, y = datasets.load_diabetes(return_X_y=True)
K = 10
kf = KFold(n_splits=K, shuffle=True, random_state=0)

hy = np.empty_like(y)
for tr, ts in kf.split(X, y):
    hy[ts] = LinearRegression().fit(X[tr], y[tr]).predict(X[ts])

B = []
for _ in range(1000):
    s = np.random.randint(X.shape[0], size=X.shape[0])
    f = r2_score(y[s], hy[s])
    B.append(f)
alpha = 0.05 / 2
Cn = (np.percentile(B, alpha * 100, axis=0), np.percentile(B, (1 - alpha) * 100, axis=0))
Cn

(0.426501013653632, 0.5562221109048776)

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
import numpy as np

# Configurar el número de splits (K) y alpha
K = 10
alpha = 0.05

# Cargar datos de Diabetes
X, y = load_diabetes(return_X_y=True)

# Crear un modelo de regresión lineal
model = LinearRegression()

# Calcular R^2 utilizando validación cruzada
y_true, y_pred = [], []

for tr, ts in KFold(n_splits=K, shuffle=True, random_state=0).split(X, y):
    model.fit(X[tr], y[tr])
    y_true.extend(y[ts])
    y_pred.extend(model.predict(X[ts]))

r2_value = r2_score(y_true, y_pred)

# Calcular el intervalo de confianza
lower_bound = r2_value - 1.96 * np.std(y_pred) / np.sqrt(len(y_pred))
upper_bound = r2_value + 1.96 * np.std(y_pred) / np.sqrt(len(y_pred))

# Imprimir el intervalo de confianza superior con dos decimales
print(f"El valor del intervalo superior del R^2 es: {upper_bound:.2f}")


El valor del intervalo superior del R^2 es: 5.67


In [63]:
upper_bound

5.673394876038918